<a href="https://colab.research.google.com/github/yutakasawai/Pytorch_sandbox/blob/master/%E8%BB%A2%E7%A7%BB%E5%AD%A6%E7%BF%92CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip list | grep cata

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install -U catalyst

     |████████████████████████████████| 398 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 308 kB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.7 MB/s  eta 0:00:01


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [5]:
import os
import random

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from torchvision import datasets,transforms,models
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from catalyst import dl
from catalyst.utils import metrics
from catalyst.dl.callbacks import AccuracyCallback
from catalyst.core.callbacks.early_stop import EarlyStoppingCallback

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

    
seed_everything(1192)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
val_transform = transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
])

train_transform = transforms.Compose([
        transforms.Resize([224,224]),
        transforms.RandomCrop(224,padding=4),
        transforms.RandomHorizontalFlip(),

        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
])

train_dataset = datasets.CIFAR10(root="./data",train=True,download=True,transform=train_transform)
val_dataset = datasets.CIFAR10(root="./data",train=False,download=True,transform=val_transform)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# val_transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.5,),(0.5,))
# ])

# train_transform = transforms.Compose([
#         # transforms.RandomHorizontalFlip(),
#         # transforms.ColorJitter(),
#         # transforms.RandomRotation(10),
#         transforms.ToTensor(),
#         transforms.Normalize((0.5,),(0.5,))
# ])

# train_dataset = datasets.CIFAR10(root="./data",train=True,download=True,transform=train_transform)
# val_dataset = datasets.CIFAR10(root="./data",train=False,download=True,transform=val_transform)

In [10]:
class ResNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        trained_model = models.resnet18(pretrained=True)
        self.trained_model = nn.Sequential(*list(trained_model.children())[:-1])
        in_features = trained_model.fc.in_features
        self.drop = nn.Dropout()
        self.final = nn.Linear(in_features,10)
    def forward(self,x):
        x = self.trained_model(x)
        x = self.drop(x.view(-1,self.final.in_features))
        x = self.final(x)
        return x

In [11]:
model = ResNetModel().to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [12]:
print(model)

ResNetModel(
  (trained_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [13]:
# for param in model.parameters():
#     param.requires_grad = False

In [14]:
for x,y in model.named_children():
    print(x)

trained_model
drop
final


In [15]:
print(model)

ResNetModel(
  (trained_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [16]:
names = ("plane","car","bird","cat","deer","dog","frog","horse","ship","truck")

In [17]:
use_cuda = torch.cuda.is_available()
kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

BATCH_SIZE = 128
train_dataloader = DataLoader(train_dataset,batch_size= BATCH_SIZE ,shuffle=True, **kwargs)
val_dataloader = DataLoader(val_dataset,batch_size= BATCH_SIZE ,shuffle=False, **kwargs)


In [18]:
criterion = nn.CrossEntropyLoss()
param_groups = [{'params':model.trained_model.parameters(),'lr':0.0001},
                {'params':model.final.parameters(),'lr':0.001}]
#optimizer = optim.Adam(model.fc.parameters(),lr=0.001)
optimizer = optim.Adam(param_groups)

In [19]:
import collections
loaders = collections.OrderedDict()
loaders["train"]=train_dataloader
loaders["valid"]=val_dataloader

In [20]:
from catalyst.dl.runner import SupervisedRunner
runner = SupervisedRunner(device="cuda")

In [21]:
# for i,l in train_dataloader:
#     print(i,l)
# i.size()

In [22]:
!rm -rf ./logs

In [ ]:
runner.train(model=model,
            optimizer=optimizer,
             criterion=criterion,
            loaders=loaders,
            callbacks=[AccuracyCallback(num_classes=10,accuracy_args=[1])],
            main_metric="accuracy01",minimize_metric=False,
             logdir="./logs",
             num_epochs=5,
             verbose=True,
             
             
            )

1/5 * Epoch (train): 100% 391/391 [02:06<00:00,  3.09it/s, accuracy01=0.875, loss=0.319]
1/5 * Epoch (valid): 100% 79/79 [00:12<00:00,  6.13it/s, accuracy01=1.000, loss=0.065]
[2020-08-01 23:50:36,564] 
1/5 * Epoch 1 (_base): lr=0.0001 | momentum=0.9000
1/5 * Epoch 1 (train): accuracy01=0.8712 | loss=0.3768
1/5 * Epoch 1 (valid): accuracy01=0.9230 | loss=0.2260
2/5 * Epoch (train):  21% 81/391 [00:26<01:41,  3.05it/s, accuracy01=0.945, loss=0.243]

In [ ]:
num_epochs = 10
losses  =[]
accs = []
val_losses = []
val_accs  = []

for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc =0.0
    for imgs,labels in train_dataloader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = model(imgs)

        loss = criterion(output,labels)
        loss.backward()
        running_loss += loss.item()
        pred = torch.argmax(output,dim=1)
        running_acc += torch.mean(pred.eq(labels).float())
        optimizer.step()

    running_loss /= len(train_dataloader)
    running_acc /= len(train_dataloader)
    
    losses.append(running_loss)
    accs.append(running_acc)

    val_runngin_loss = 0.0
    val_running_acc = 0.0

    for val_imgs,val_labels in val_dataloader:
        val_imgs = val_imgs.to(device)
        val_labels = val_labels.to(device)

        val_output = model(val_imgs)
        val_loss = criterion(val_output, val_labels)

        val_runngin_loss += val_loss.item()
        val_pred = torch.argmax(val_output,dim=1)
        val_running_acc += torch.mean(val_pred.eq(val_labels).float())
    val_runngin_loss/= len(val_dataloader)
    val_running_acc /= len(val_dataloader)

    val_losses.append(val_runngin_loss)
    val_accs.append(val_running_acc)

    print("epoch :{}, loss:{} ,acc:{},val loss :{} ,val acc :{}".format(epoch,running_loss,running_acc,val_runngin_loss,val_running_acc))

In [ ]:
plt.style.use("ggplot")
plt.plot(losses,label="train loss")
plt.plot(val_losses,label="val loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(accs,label ="train acc")
plt.plot(val_accs,label="val acc")
plt.legend()